In [1]:
tags = [
    'us', 'spain', 'italy', 'france', 'germany', 'uk', 'turkey', 'iran', 
    'russia', 'brazil', 'canada', 'belgium', 'netherlands', 'switzerland', 
    'india', 'peru', 'portugal', 'ecuador', 'ireland', 'sweden', 'saudi-arabia', 
    'israel', 'austria', 'mexico', 'singapore', 'japan', 'chile', 'pakistan', 
    'poland', 'romania', 'belarus', 'qatar', 'south-korea', 'united-arab-emirates', 
    'indonesia', 'ukraine', 'denmark', 'serbia', 'philippines', 'norway', 
    'czech-republic', 'australia', 'dominican-republic', 'bangladesh', 'malaysia', 
    'panama', 'colombia', 'finland', 'south-africa', 'egypt', 'morocco', 'argentina', 
    'luxembourg', 'moldova', 'algeria', 'kuwait', 'thailand', 'kazakhstan', 
    'bahrain', 'hungary', 'greece', 'oman', 'croatia', 'uzbekistan', 'iraq', 
    'armenia', 'iceland', 'afghanistan', 'estonia', 'azerbaijan', 'cameroon', 
    'bosnia-and-herzegovina', 'ghana', 'new-zealand', 'lithuania', 'slovenia', 
    'macedonia', 'slovakia', 'cuba', 'bulgaria', 'nigeria', 'cote-d-ivoire', 
    'china-hong-kong-sar', 'djibouti', 'guinea', 'bolivia', 'tunisia', 'latvia', 
    'cyprus', 'andorra', 'albania', 'lebanon', 'niger', 'kyrgyzstan', 'costa-rica', 
    'senegal', 'honduras', 'burkina-faso', 'uruguay', 'sri-lanka', 'san-marino', 
    'channel-islands', 'guatemala', 'georgia', 'democratic-republic-of-the-congo', 
    'malta', 'jordan', 'somalia', 'taiwan', 'reunion', 'mayotte', 'mali', 'kenya', 
    'jamaica', 'state-of-palestine', 'mauritius', 'venezuela', 'el-salvador', 
    'montenegro', 'isle-of-man', 'tanzania', 'viet-nam', 'equatorial-guinea', 
    'sudan', 'paraguay', 'maldives', 'congo', 'rwanda', 'faeroe-islands', 'gabon', 
    'martinique', 'guadeloupe', 'myanmar', 'gibraltar', 'brunei-darussalam', 
    'liberia', 'ethiopia', 'madagascar', 'cambodia', 'trinidad-and-tobago', 
    'french-guiana', 'bermuda', 'cabo-verde', 'aruba', 'togo', 'monaco', 
    'sierra-leone', 'zambia', 'liechtenstein', 'bahamas', 'barbados', 'uganda', 
    'mozambique', 'sint-maarten', 'guyana', 'haiti', 'cayman-islands', 'benin', 
    'libya', 'swaziland', 'french-polynesia', 'guinea-bissau', 'nepal', 'chad', 
    'china-macao-sar', 'syria', 'eritrea', 'saint-martin', 'mongolia', 'malawi', 
    'zimbabwe', 'angola', 'antigua-and-barbuda', 'timor-leste', 'botswana', 
    'central-african-republic', 'laos', 'belize', 'fiji', 'grenada', 'new-caledonia', 
    'curacao', 'dominica', 'namibia', 'saint-kitts-and-nevis', 'saint-lucia', 
    'saint-vincent-and-the-grenadines', 'nicaragua', 'falkland-islands-malvinas', 
    'burundi', 'montserrat', 'turks-and-caicos-islands', 'greenland', 'seychelles', 
    'gambia', 'suriname', 'holy-see', 'papua-new-guinea', 'mauritania', 'bhutan', 
    'british-virgin-islands', 'saint-barthelemy', 'south-sudan', 'western-sahara', 
    'caribbean-netherlands', 'sao-tome-and-principe', 'anguilla', 'saint-pierre-and-miquelon', 
    'yemen', 'china'
]

In [2]:
import requests

In [3]:
from lxml.html import fromstring

In [4]:
import re

In [5]:
lines = re.compile(r'[\n\t]')

In [6]:
comments = re.compile(r'\/\*(.*?)\*\/')

In [7]:
spaces = re.compile(r'[ ]{2,}')

In [8]:
chart = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )(.*?)(\}\)\;)')

In [9]:
chart_title = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )')

In [10]:
chart_end = re.compile(r'\)\;')

In [11]:
options = re.compile(r' [\w]+\: ')

In [12]:
xaxis = re.compile(r'(\"xAxis\"\: \{)(.*?)(\}\,)')

In [13]:
yaxis = re.compile(r'(\"yAxis\"\: \{ \"title\"\: \{ \"text\"\: \")(.*?)(\")')

In [14]:
series = re.compile(r'(\"series\"\: \[\{)(.*?)(\}( ){0,}\]\,)')

In [15]:
import json

In [16]:
from datetime import datetime

In [17]:
import pandas as pd

In [18]:
import os

In [19]:
for tag in tags:
    url = f"https://www.worldometers.info/coronavirus/country/{tag}/"
    
    print(f"Retrieving Coronavirus data from Country with Tag: {tag}")
    
    req = requests.get(url)
    
    if req.status_code != 200:
        continue
        
    root = fromstring(req.text)
    scripts = root.xpath(".//script")
    
    for script in scripts:
        if not script.text_content().strip().__contains__("Highcharts.chart"):
            continue
            
        value = script.text_content().strip()
        real_value = script.text_content().strip()
        
        value = lines.sub('', value)
        value = comments.sub('', value)
        value = spaces.sub(' ', value)
        value = value.replace('\'', '\"')
        value = chart.search(value).group(0)
        
        title = chart_title.search(value).group(0).replace('Highcharts.chart(\"', '').replace('\", ', '')
        
        value = chart_title.sub('', value)
        value = chart_end.sub('', value)

        words = options.findall(value)

        for word in words:
            value = value.replace(word, ' \"' + word.replace(' ', '').replace(':', '') + '\": ')
            
        column = yaxis.search(value).group(0).replace('"yAxis": { "title": { "text": "', '').replace('"', '')
        
        value = '{' + ', '.join([
            xaxis.search(value).group(0).replace('},', '}'),
            series.search(value).group(0)[::-1].replace(',]', ']', 1)[::-1].replace(', }', ' }')
        ]) + '}'
        
        value = json.loads(value)

        x = value['series'][0]['data']

        y = [datetime.strptime(val + ', 2020', '%b %d, %Y') for val in value['xAxis']['categories']]

        data = pd.DataFrame({'Date': y, column: x})
        data.set_index('Date', inplace=True)
        
        path = f'../data/{tag}'
        if not os.path.exists(path):
            os.makedirs(path)

        data.to_csv(path + '/' + title + '.csv')

Retrieving Coronavirus data from Country with Tag: us
Retrieving Coronavirus data from Country with Tag: spain
Retrieving Coronavirus data from Country with Tag: italy
Retrieving Coronavirus data from Country with Tag: france
Retrieving Coronavirus data from Country with Tag: germany
Retrieving Coronavirus data from Country with Tag: uk
Retrieving Coronavirus data from Country with Tag: turkey
Retrieving Coronavirus data from Country with Tag: iran
Retrieving Coronavirus data from Country with Tag: russia
Retrieving Coronavirus data from Country with Tag: brazil
Retrieving Coronavirus data from Country with Tag: canada
Retrieving Coronavirus data from Country with Tag: belgium
Retrieving Coronavirus data from Country with Tag: netherlands
Retrieving Coronavirus data from Country with Tag: switzerland
Retrieving Coronavirus data from Country with Tag: india
Retrieving Coronavirus data from Country with Tag: peru
Retrieving Coronavirus data from Country with Tag: portugal
Retrieving Coro